# Gradients through iLQR

## Imports

In [4]:
from functools import partial
from pathlib import Path
import chex
import jax
from jax import Array
from jax import tree_map
import jax.random as jr
import jax.numpy as jnp
import numpy as onp
from matplotlib import pyplot as plt
from diffilqrax.diff_ilqr import dilqr
from diffilqrax.ilqr import ilqr_solver
from diffilqrax.exact import quad_solve, exact_solve
from diffilqrax.utils import keygen
from diffilqrax.typs import (
    iLQRParams,
    System,
    ModelDims,
    Thetax0,
    Theta,
)

jax.config.update("jax_default_device", jax.devices("cpu")[0])
jax.config.update("jax_enable_x64", True)  # double precision


/var/folders/gn/k1gxpths3215w28g0srkkq7r0000gn/T/ipykernel_22126/3780710678.py:6: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  from jax import tree_map


## iLQR Optimisation Problem

The Iterative Linear Quadratic Regulator (iLQR) is an optimization algorithm used for trajectory optimization in control systems. It iteratively refines a control policy to minimize a cost function over a finite time horizon.

### Problem Formulation

Given a dynamical system described by the state equation:
$$ x_{t+1} = f(x_t, u_t) $$
where  $x_t$ is the state at time $t$ and $u_t$ is the control input at time $ t $.

The objective is to find a sequence of control inputs $ \{u_0, u_1, \ldots, u_{T-1}\} $ that minimizes the cost function:
$$ J = \ell_{T}(x_T) + \sum_{t=0}^{T-1} \ell(x_t, u_t) $$
where $ \ell_{T}(x_T) $ is the terminal cost and $ \ell(x_t, u_t) $ is the running cost.

### Linear Quadratic Approximation

iLQR approximates the nonlinear dynamics and cost function using a quadratic expansion around a nominal trajectory $ \{x_t^0, u_t^0\} $:

1. **Dynamics Linearization:**
    $$ x_{t+1} \approx A_t x_t + B_t u_t + c_t $$
    where $ A_t = \frac{\partial f}{\partial x} \bigg|_{(x_t^0, u_t^0)} $, $ B_t = \frac{\partial f}{\partial u} \bigg|_{(x_t^0, u_t^0)} $, and $ c_t = f(x_t^0, u_t^0) - A_t x_t^0 - B_t u_t^0 $.

2. **Cost Quadratic Approximation:**
    $$ \ell(x_t, u_t) \approx \frac{1}{2} \begin{bmatrix} x_t \\ u_t \end{bmatrix}^T \begin{bmatrix} Q_t & S_t \\ S_t^T & R_t \end{bmatrix} \begin{bmatrix} x_t \\ u_t \end{bmatrix} + \begin{bmatrix} q_t \\ r_t \end{bmatrix}^T \begin{bmatrix} x_t \\ u_t \end{bmatrix} + l_t $$
    where $ Q_t, S_t, R_t, q_t, r_t, l_t $ are the coefficients of the quadratic expansion.

### Backward Pass

In the backward pass, the value function is approximated by a quadratic function:
$$ V_t(x_t) \approx \frac{1}{2} x_t^T P_t x_t + p_t^T x_t + \alpha_t $$

The optimal control law is derived as:
$$ \delta u_t = -K_t \delta x_t - k_t $$
where $ K_t $ and $ k_t $ are the feedback and feedforward gains, respectively.

### Forward Pass

In the forward pass, the control inputs are updated using the derived control law, and the state trajectory is propagated using the system dynamics.

### Iteration

The process of linearization, backward pass, and forward pass is repeated until convergence.

This iterative process refines the control policy and trajectory, leading to an optimal solution for the given cost function and system dynamics.


## Set-up problem

### Implementation

The iLQR `System` contains the dimensionality of the problem in `ModelDims`, the non-linear dynamics, cost, and terminal cost functions.

In [5]:
# Set-up dimensionality
dims = ModelDims(horizon=100, n=10, m=2, dt=0.1)

# Set-up prarameters
key = jr.PRNGKey(seed=0)
key, skeys = keygen(key, 3)
Uh = jax.random.normal(key, (dims.n, dims.n)) * 0.5 / jnp.sqrt(dims.n)

Wh =  jax.random.normal(key, (dims.n, dims.m)) * 0.5 / jnp.sqrt(dims.n) * dims.dt
L = jax.random.normal(key, (dims.n, dims.n)) * 0.5 / jnp.sqrt(dims.n)
Q = L @ L.T

In [7]:
# Define iLQR model
def cost(t: int, x: Array, u: Array, theta: Theta):
    x_tgt = jnp.sin(t/5)
    return (
        jnp.sum(
            (x.squeeze() - x_tgt.squeeze())
            @ theta.Q @ 
            (x.squeeze() - x_tgt.squeeze()).T
        )
        + jnp.sum(u**2)
    )

def costf(x: Array, theta: Theta):
    # return jnp.sum(jnp.abs(x))
    return jnp.sum(x**2)

def dynamics(t: int, x: Array, u: Array, theta: Theta):
    return (theta.Uh @ jnp.tanh(x)) + theta.Wh @ jnp.tanh(u) + jnp.sum(theta.Uh)*jnp.ones_like(x)

ilqr_model = System(
    cost, costf, dynamics, dims
)


## Solve iLQR

## iLQR Gradient